In [1]:
from bs4 import BeautifulSoup

In [2]:
def extract_data_from_report3(filename):
    html_report_part1 = open(filename,'r')
    soup = BeautifulSoup( html_report_part1, "html.parser")
    return soup

In [3]:
def GetInfo(htmFile):
    dic = {}
    sup = extract_data_from_report3(htmFile)
    for tag in sup.find_all("meta"):
        name = tag.get("name")
        if(name=="Description"):
            Description = tag.get("content")
            dic["Description"] = Description
        if(name=="System.Keywords"): 
            Keywords = tag.get("content")
            dic["Keywords"] = Keywords
        if(name=="Microsoft.Help.F1"): 
            NameSpace = tag.get("content")
            dic["NameSpace"] = NameSpace
        if(name=="guid"): 
            Guid = tag.get("content")
            dic["Guid"] = Guid
        if(name=="container"):
            ClassName = tag.get("content")
            dic["ClassName"] = ClassName
    return list(dic.values()) 

In [4]:
 # input folder containt all file htm extracted from file chm guide
folder = r"C:\Users\Chuong.Ho\3D Objects\properties"
import os
datas = []
def ReadFilesFolder(folder):
    files = []
    for file in os.listdir(folder):
        if file.endswith(".htm"):
            files.append(folder+"\\"+file)
    return files
for file in ReadFilesFolder(folder):
    try:
        metadata = GetInfo(file)
        datas.append(metadata)
    except Exception as e:
        print(e)
        continue

'charmap' codec can't decode byte 0x9d in position 11591: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 863: character maps to <undefined>
'charmap' codec can't decode byte 0x8f in position 835: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 24276: character maps to <undefined>
'charmap' codec can't decode byte 0x8f in position 833: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 863: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 856: character maps to <undefined>
'charmap' codec can't decode byte 0x8f in position 86366: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 856: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 863: character maps to <undefined>
'charmap' codec can't decode byte 0x81 in position 863: character maps to <undefined>
'charmap' codec can't decode byte 0x8f in positi

In [10]:
from pandas import DataFrame
header = ["Keywords","NameSpace","Description","ClassName","Guid"]
df = DataFrame(datas,columns=header,dtype=str)
df = df.dropna()
df.head(10)

,Keywords,NameSpace,Description,ClassName,Guid
1,ParameterTypeId.BoundaryRestraintRotZ property,Autodesk.Revit.DB.ParameterTypeId.get_Boundary...,"""Z Spring Modulus""",Autodesk.Revit.DB,0002e99b-f1c0-0480-514a-c3d9f466b9fd
2,ElectricalDemandFactorValue.MaxRange property,Autodesk.Revit.DB.Electrical.ElectricalDemandF...,The maximum range for this demand factor value...,Autodesk.Revit.DB.Electrical,0003f9f7-1be6-9bcb-212a-d38767d01e10
3,StructuralAsset.ConcreteBendingReinforcement p...,Autodesk.Revit.DB.StructuralAsset.set_Concrete...,The bending reinforcement of the asset.,Autodesk.Revit.DB,0005810b-32f5-0623-b094-c0c47fb50d32
4,BuiltInFailures.StairRampFailures.StairsZeroDe...,Autodesk.Revit.DB.BuiltInFailures.StairRampFai...,The number of desired risers for a staircase c...,Autodesk.Revit.DB,00077047-bcc4-4b8f-8a52-07340ec04412
5,SymbolTypeId.Kip property,Autodesk.Revit.DB.SymbolTypeId.get_Kip,"Symbol kip, indicating unit Kips.",Autodesk.Revit.DB,00095a9c-e1f8-8edf-ff9b-dbb512951943
6,AreaLoad.ForceVector3 property,Autodesk.Revit.DB.Structure.AreaLoad.set_Force...,The force vector applied to the 3rd reference ...,Autodesk.Revit.DB.Structure,000a344f-d68f-8718-c6e5-72edef5ce04c
8,SymbolTypeId.NPerMmSup2 property,Autodesk.Revit.DB.SymbolTypeId.get_NPerMmSup2,"Symbol N/mmÃ‚Â², indicating unit Newtons per s...",Autodesk.Revit.DB,000ea288-c477-86a8-57e0-764f2e359d5b
9,ExternalResourceTypes.BuiltInExternalResourceT...,Autodesk.Revit.DB.ExternalResourceTypes.BuiltI...,An external resource type representing Revit L...,Autodesk.Revit.DB,0010ac27-ce75-aae5-af64-0f58a6411109
10,MetallicPaint.MetallicpaintBaseColorByObject p...,Autodesk.Revit.DB.Visual.MetallicPaint.get_Met...,"The property labeled ""Color By Object"" from th...",Autodesk.Revit.DB.Visual,00135055-9bf2-2f0c-cb5b-067ab92d922d
13,FamilyItemFactory.NewBlend method,Autodesk.Revit.Creation.FamilyItemFactory.NewB...,Add a new Blend instance into the Autodesk Rev...,Autodesk.Revit.Creation,00173a0d-6075-0270-fd7e-080da420d339


In [6]:
#save dataframe to json file
df.to_json("RevitAPI.json")